In [1]:
import importlib

In [2]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [3]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from datasets import load_sift_dataset, calculate_recall

In [20]:
hnsw_a = HNSW( distance_func=l2_distance, m=16, m0=32, ef=32, ef_construction=32,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=16, m0=32, ef=32, ef_construction=32,  neighborhood_construction = heuristic)

In [21]:
hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

In [22]:
merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [23]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import merge1

In [24]:
%%time
distance_count = 0

jump_ef=20
local_ef=5
next_step_k=5
next_step_ef=3
M = 5
hnsw_merged1 = merge1(hnsw_a, hnsw_b, merged_data, jump_ef=jump_ef, local_ef=local_ef, next_step_k=next_step_k, next_step_ef=next_step_ef, M = M)

Merging level: 0


Merging progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [54:09<00:00, 153.85it/s]


Merging level: 1


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31358/31358 [01:33<00:00, 337.00it/s]


Merging level: 2


...

Merging level: 3


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 538.88it/s]


Merging level: 4


Merging progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1677.05it/s]


CPU times: user 1h 37min 11s, sys: 14min 51s, total: 1h 52min 3s
Wall time: 1h 47min 47s


In [9]:
hnsw_merged1.save(f'save/sift1m/hnsw_merged1_jef{jump_ef}_lef{local_ef}_nsk{next_step_k}_nsef{next_step_ef}_M{M}.txt')

In [25]:
print(f'Numer of distance calculated during merge: {distance_count}') # m=5 m0=7: 695004299, m=16, m0=32: 947762326

Numer of distance calculated during merge: 947762326


In [14]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [26]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:44<00:00, 95.63it/s]

32,0.8989599999999999, 920.6024


In [27]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:19<00:00, 71.44it/s]

40,0.9209200000000001, 1062.3704


In [28]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:39<00:00, 35.75it/s]

64,0.95336, 1468.1675


In [30]:
efs=[50,72]
# efs=[32,40,50,64,72]
for ef in efs:
    distance_count = 0
    recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=ef)
    print(f'ef={ef} recall: {recall}, avg dist: {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [03:28<00:00, 47.90it/s]


ef=50 recall: 0.9381200000000001, avg dist: 1234.3287
Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:20<00:00, 31.23it/s]

ef=72 recall: 0.9596199999999999, avg dist: 1598.5293
